In [374]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas_datareader import data as wb
from datetime import datetime
from dateutil.relativedelta import relativedelta
from dateutil.parser import parser
from functools import cached_property, partial
import plotly.express as px
import re

class DataManipulation:
    def _subset_by_dates(self, df, start_date=None, end_date=None):
        if end_date is None:
            end_date = datetime.today()
        else:
            end_date = parser().parse(end_date)
        if start_date is None:
            start_date = end_date-relativedelta(years=5)
        else:
            start_date = parser().parse(start_date)
        return df[(df.index >= start_date)&(df.index <= end_date)]
    
    def _calculate_returns(self, df, start_date=None, end_date=None, log=False):
        df = self._subset_by_dates(df, start_date, end_date)
        if log is True:
            return np.log(df.pct_change()+1)
        else:
            return df.pct_change()
    
    def _get_market_data(self, start_date=None, end_date=None, market_ticker="^GSPC"):
        return wb.DataReader(market_ticker, data_source='yahoo',
                                             start='1970-1-1')['Close']
    
    def _lookup_values(self, df, column, value, concol, condition="eq", to_="array"):
        is_col_index = True if (column == df.index.name)|(column == "index") else False
        if is_col_index:
            if condition == "gt":
                sub = df[df.index>=value]
            elif condition == "lt":
                sub = df[df.index<=value]
            else:
                sub = df[df.index==value]
        else:
            if condition == "gt":
                sub = df[df[column]>=value]
            elif condition == "lt":
                sub = df[df[column]<=value]
            else:
                sub = df[df[column]==value]
                
        if to_ == "array":
            if is_col_index:
                return sub.index.values
            else:
                return sub[column].values
        elif to_ == "df":
            return sub
        elif to_ == "series":
            if is_col_index:
                return sub.index
            else:
                return sub[column]
        elif to_ == "single":
            if is_col_index:
                return sub.index.values[0]
            else:
                return sub[column].values[0]
        else:
            if is_col_index:
                return list(sub.index.values)
            else:
                return list(sub[column].values)
        
        

In [375]:
dm = DataManipulation()
dm._lookup_values(investment_transactions, "Qty", 20, "gt", "df")

array([20], dtype=int64)

In [376]:
class Stock(DataManipulation):
    def __init__(self,ticker):
        self.symbol = ticker
        self.historical_data = pd.DataFrame()
        self.historical_data[ticker] = wb.DataReader(self.symbol, data_source='yahoo',
                                             start='1970-1-1')['Close']
    
    def volatility(self):
        # Assumes 5 year volatility
        five_years_ago = (datetime.today()-relativedelta(years=5)).strftime("%Y-%m-%d")
        tday = datetime.today().strftime("%Y-%m-%d")
        temp_df = self._subset_by_dates(self.historical_data, 
                                   five_years_ago, tday)
        temp_var = temp_df.values.std()
        return temp_var
    
    def plot(self, start_date=None, end_date=None):
        temp_df = self._subset_by_dates(self.historical_data, start_date, end_date)
        fig = px.line(temp_df, title=f"{self.symbol} Stock Price")
        return fig.show()
    
    def plot_returns(self, start_date=None, end_date=None):
        temp_df = self._calculate_returns(
                    self._subset_by_dates(
                        self.historical_data, start_date, end_date))
        return px.line(temp_df).show()
            
    def compare_to_market(self, start_date=None, end_date=None, market_ticker="^GSPC"):
        market_df = pd.DataFrame()
        market_df[market_ticker] = self._subset_by_dates(self._get_market_data(market_ticker),start_date,end_date)
        df = self._subset_by_dates(self.historical_data, start_date, end_date) 
        mx = df.join(market_df)
        temp_df = mx/mx.iloc[0]
        return px.line(temp_df, labels={'x':market_ticker}).show()
    
    def _moving_average(self, *days, start_date=None, end_date=None):
        temp_df = self._subset_by_dates(self.historical_data, start_date, end_date)
        
        for d in days:
            temp_df[f"MA {d} days"] = temp_df[self.symbol].rolling(d, min_periods=1).mean()
        return px.line(temp_df)
                
        

In [377]:
abmd = Stock("ABMD")

In [378]:
abmd._moving_average(30,60,90,250,500,750,1000, end_date="2020-01-01")

<ipython-input-376-f48dee728c0b>:40: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [241]:
transactions_path = "https://s3.amazonaws.com/www.yourdatasurfer.com/test_stock_transactions.csv"
investment_transactions = pd.read_csv(transactions_path)#.rename(columns={"Qty":"Quantity", "Open date":"Date"})
investment_transactions['Open date'] = pd.to_datetime(investment_transactions['Open date'])
investment_transactions['Cost_per_share'] = investment_transactions['Adj cost per share']
investment_transactions.head()

Symbol           Security  Qty Type  Open date  Adj cost per share  \
0   AMZN     AMAZON COM INC    4  Buy 2014-01-06              390.87   
1     FB  FACEBOOK INC CL A   60  Buy 2014-01-06               54.79   
2    GLW        CORNING INC  100  Buy 2014-01-06               17.85   
3   GOOG  ALPHABET INC-CL C    2  Buy 2014-01-06              554.58   
4  GOOGL  ALPHABET INC-CL A    2  Buy 2014-01-06              557.89   

   Adj cost  Cost_per_share  
0   1563.48          390.87  
1   3287.40           54.79  
2   1784.99           17.85  
3   1109.16          554.58  
4   1115.77          557.89

In [287]:
class Position(DataManipulation):
    def __init__(self, Stock):
        self._stock = Stock
        self._symbol = Stock.symbol
        self._historical_stock_data = self._stock.historical_data
        self.shares_owned = 0
        self.invested = 0
        self.market_value = 0
        self.transaction_history = pd.DataFrame(columns=["Symbol","Qty","Date","Type","Cost_per_share"])
        
    def buy(self, quantity, date, price=None):
        if price is None:
            price = self._subset_by_dates(self._historical_stock_data, date, date)[self._symbol].values[0]
        self.shares_owned += quantity
        self.invested += quantity*price
        pass
    
    def update(self, date):
        pass
    
    def sell(self, quantity, date, price=None):
        if price is None:
            print(f"You do not own enough shares of {self._stock.symbol} to sell {quantity} shares on {date}.\nPosition will be updated.")
            return self.update(date)
        
    def load_transaction_history(self, path):
        # ASsume column names are correct:
        ## Symbol, Security, Qt, Type, Open date, Adj cost per share, Adj cost
        
        # Read pandas from path / csv / Excel
        filetype = re.findall("\.\w{2,4}$",path)[0]
        if filetype == ".csv":
            temp_df = pd.read_csv(path)
        elif filetype == [".xlsx",".xlsm",".xls"]:
            temp_df = pd.read_excel(path)
        else:
            print(f"LoadError: are you sure the path {path} is correct? Make sure your file is in the correct format.")
        
        # Include basic statistics
        temp_df['Open date'] = pd.to_datetime(temp_df['Open date'])
        temp_df['Cost_per_share'] = temp_df['Adj cost per share']
        temp_df = temp_df[temp_df['Symbol']==self._stock.symbol].reset_index(drop=True)
        temp_df['Date'] = pd.to_datetime(temp_df['Open date'])
        temp_df = temp_df[['Symbol','Qty','Date','Type','Cost_per_share']]
        
        # Minimum date
        first_buy = min(temp_df['Date'])
        diary_df = self._lookup_values(self._historical_stock_data, "index", first_buy, "gt", "df")
        print(diary_df)
        shares_owned = 0
        for i, row in temp_df.iterrows():
            if row['Type'] == "Buy":
                shares_owned = shares_owned + row['Qty']
            elif row['Type'] == "Sell":
                shares_owned = shares_owned - row['Qty']
            else:
                print("Unrecognized transaction type")
            temp_df.at[i, "Owned_Shares"] = shares_owned
        
        self.transaction_history = diary_df
        
            

In [288]:
aapl = Stock("AAPL")

In [289]:
aapl.historical_data.head()

AAPL
Date                
1980-12-12  0.128348
1980-12-15  0.121652
1980-12-16  0.112723
1980-12-17  0.115513
1980-12-18  0.118862

In [290]:
pos = Position(aapl)
pos.load_transaction_history(transactions_path)
pos.transaction_history

                  AAPL
Date                  
2015-08-26   27.422501
2015-08-27   28.230000
2015-08-28   28.322500
2015-08-31   28.190001
2015-09-01   26.930000
...                ...
2020-10-02  113.019997
2020-10-05  116.500000
2020-10-06  113.160004
2020-10-07  115.080002
2020-10-08  114.970001

[1290 rows x 1 columns]


AAPL
Date                  
2015-08-26   27.422501
2015-08-27   28.230000
2015-08-28   28.322500
2015-08-31   28.190001
2015-09-01   26.930000
...                ...
2020-10-02  113.019997
2020-10-05  116.500000
2020-10-06  113.160004
2020-10-07  115.080002
2020-10-08  114.970001

[1290 rows x 1 columns]

In [272]:
pos.buy(10, "2017-09-27")

38.557498931884766


In [296]:
pos.transaction_history.rolling(30, min_periods=1).mean().plot()